In [8]:
from gurobipy import *
from prettytable import PrettyTable
import numpy as np

In [9]:
EC = Model('Dynamic Economic Dispatch')

In [41]:
#Sets
G = 4
Gen = range(G)
T =  3# Demand per Period
Period = range(T)

GenAt = {Gen[0]:{'a':0.12 ,'b':14.80 ,'c':89, 'd':1.2 ,'e':-5 ,'f':3 , 'Pmin':28 ,'Pmax':200,'RU0':40,'RD0':40 },
      Gen[1]:{'a':0.17 ,'b':16.57 ,'c':83 , 'd':2.3 ,'e':-4.24 ,'f':6.09 , 'Pmin':20 ,'Pmax':290,'RU0':30,'RD0':30  },
      Gen[2]:{'a':0.15 ,'b':15.55 ,'c':100 , 'd':1.1 ,'e':-2.15 ,'f':5.69 ,'Pmin':30 ,'Pmax':190,'RU0':30,'RD0':30  },
      Gen[3]:{'a':0.19 ,'b':16.21 ,'c':70 , 'd':1.1 ,'e':-3.99 ,'f':6.2 ,'Pmin':20 ,'Pmax':260, 'RU0':50,'RD0':50 }  }

In [42]:
Demand = [300, 300, 300]

In [43]:
Pmin = [GenAt[i]['Pmin'] for i in Gen]
Pmax = [GenAt[i]['Pmax'] for i in Gen]
l = [(a,b) for a in Gen for b in Period]

In [44]:
P = EC.addVars(l, lb = Pmin*T, ub = Pmax*T)
RU = EC.addConstrs((P[(i, j)] - P[(i, j-1)]  <= GenAt[i]['RU0']) for i in Gen for j in range(1,T))
RD = EC.addConstrs(P[(i, j-1)] - P[(i, j)]  <= GenAt[i]['RD0'] for i in Gen for j in range(1,T))
Load = EC.addConstrs((P.sum('*',j) == Demand[j] for j in Period), name = 'DemandConst')
GenCost =quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i,j]+GenAt[i]['a']*P[i,j]*P[i,j] for i in Gen for j in Period) 
EC.setObjective(GenCost, GRB.MINIMIZE)

In [45]:
EC.setObjective(GenCost, GRB.MINIMIZE)
EC.Params.Method=-1
EC.setParam( 'OutputFlag', True )
EC.optimize()



status = EC.status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
or status == GRB.Status.UNBOUNDED:
    EC.computeIIS()
    EC.write("model.ilp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 25 rows, 36 columns and 56 nonzeros
Model has 12 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  QObjective range [2e-01, 4e-01]
  Bounds range     [1e+00, 3e+02]
  RHS range        [3e+01, 3e+02]
Presolve removed 14 rows and 24 columns
Presolve time: 0.01s
Presolved: 11 rows, 20 columns, 36 nonzeros
Presolved model has 12 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+01
 Factor NZ  : 6.600e+01
 Factor Ops : 5.060e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.57618203e+06 -3.95898091e+06  2.50e+03 8.54e+02  8.35e+05     0s
   1   9.53252736e+04 -8.68943071e+05  1.71e+02 

In [46]:
PVar = np.asarray([[P[i,j].x for i in Gen] for j in Period])
PVar

array([[99.25745017, 64.85820012, 76.90596013, 58.97838958],
       [99.25745017, 64.85820012, 76.90596013, 58.97838958],
       [99.25745017, 64.85820012, 76.90596013, 58.97838958]])